<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s overal message(disaster or not?):
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [2]:
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-29 14:12:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-29 14:12:58 (54.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-29 14:12:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2022-08-29 14:12:58 (49.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [7]:
train_data = pd.read_csv('train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
test_data = pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [10]:
random_data = train_data.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
4 equipment ego break upon dig your family internet hoke excepting versus a sinking term: dfLJEV : This is a Disaster tweet
------------------------------------

tweet number 2 :
Reddit Will Now Quarantine Offensive Content http://t.co/FEIkC9FxED #onlinecommunities #reddit /via @wired : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
Only one man Tsutomi Yamaguchi is said to have survived both atomic bomb blasts at #Hiroshima and #Nagasaki. #OTD http://t.co/DaalPeNZp0 : This is a Disaster tweet
------------------------------------

tweet number 4 :
Apparently if you're bleeding people look at you weird lol well it's fine keep walking : This is NOT Disaster tweet
------------------------------------

tweet number 5 :
@danisnotonfire if you follow me I will go into town and advertise your youtube channel by screaming and walking around with a sign. Pls?? : This is NOT Disaster tweet
------------------------------------



## Extract validation data from train data:

In [11]:
train_data['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [12]:
train_data['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [13]:
train_data, val_data, train_label, val_label = train_test_split(train_data['text'].to_numpy(), train_data['target'].to_numpy(), 
                                                                test_size=0.15, 
                                                                random_state=42)

In [14]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [15]:
sum([len(tweet.split()) for tweet in train_data])

96401

In [16]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [17]:
text_vectorization = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [18]:
text_vectorization.adapt(train_data)

In [19]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorization(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  736,    9,    1,    8,  161,  681,    5, 1732,   12,    0,
          0,    0,    0,    0])>

In [20]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorization(random_tweet))

tweet:  But the government will not care. Police will stop rioting eventually of protestors. Eventually some skyscrapers become plant-covered . 


vectorized tweet:  tf.Tensor(
[  26    2  528   42   33  619   83   42  244  384 2893    6 9140 2893
   79], shape=(15,), dtype=int64)


In [21]:
#unique words
unique_words = text_vectorization.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['nkulw', 'njenga', 'nj36', 'nittys', 'nitroglycerin', 'nitishkumar', 'nite', 'nissannews', 'nissan', 'niqqa']


### Embedding:

In [24]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim = 16, input_length=None)

In [25]:
sample_embed = embedding(text_vectorization(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 16), dtype=float32, numpy=
array([[-0.00985227,  0.03285548, -0.0126789 ,  0.04357178,  0.01940377,
        -0.02302632,  0.03870488,  0.04311714, -0.02157381,  0.03922443,
        -0.03262784, -0.02943047,  0.00827809, -0.04277639,  0.04680932,
         0.02975187],
       [-0.02591496, -0.02018712, -0.0110026 ,  0.02692102,  0.03386444,
        -0.02714635,  0.01903994,  0.02356566, -0.04404641, -0.0196939 ,
        -0.03112025,  0.0207714 , -0.00696577,  0.01531838, -0.00718804,
        -0.04671097],
       [-0.02659901,  0.02068075, -0.04080368,  0.0376912 ,  0.0240276 ,
         0.03605156, -0.01223215,  0.01380518, -0.00484233,  0.03394741,
        -0.00750374, -0.04715262, -0.00574627,  0.0257149 ,  0.01767467,
         0.03814912],
       [-0.00839338,  0.01637824,  0.03551256,  0.04273114,  0.01707704,
         0.03465061, -0.0306294 ,  0.00258965, -0.0430808 , -0.03561977,
         0.02399598, -0.03066056,  0.04918108, -0.01053536, -0.01170863,
        